In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json

In [2]:
service = Service(executable_path='./chromedriver-linux64/chromedriver')
driver = webdriver.Chrome(service=service)
driver.get('https://www.oxfordreference.com/display/10.1093/acref/9780190245115.001.0001/acref-9780190245115?btog=chap&hide=true&pageSize=20&skipEditions=true&sort=titlesort&source=%2F10.1093%2Facref%2F9780190245115.001.0001%2Facref-9780190245115')
time.sleep(6)

# Get all links

In [10]:
base_url = "https://www.oxfordreference.com/display/10.1093/acref/9780190245115.001.0001/acref-9780190245115?btog=chap&hide=true&page={}&pageSize=20&skipEditions=true&sort=titlesort&source=%2F10.1093%2Facref%2F9780190245115.001.0001%2Facref-9780190245115"

# List to hold all the extracted links
all_links = []

# Loop through all pages
for page_number in range(1, 4171):
    # Construct the URL for the current page
    url = base_url.format(page_number)
    driver.get(url)

    # Wait for the page to load
    time.sleep(3)  # Consider using WebDriverWait here for better performance

    # Find all the div elements with the specified class
    entries = driver.find_elements(By.CSS_SELECTOR, "div.contentItem.oxencycl-entry.locked.hasCover.chunkResult.hi-visible.p-4.border-top")

    # Extract links from the current page
    for entry in entries:
        link = entry.find_element(By.CSS_SELECTOR, "h2.itemTitle a").get_attribute("href")
        all_links.append(link)

    # Optional: print or log the progress
    print(f"Processed page {page_number}")

# After finishing all pages, print the number of collected links
print(f"Total links collected: {len(all_links)}")

# Close the browser
driver.quit()


Processed page 1
Processed page 2
Processed page 3
Processed page 4
Processed page 5
Processed page 6
Processed page 7
Processed page 8
Processed page 9
Processed page 10
Processed page 11
Processed page 12
Processed page 13
Processed page 14
Processed page 15
Processed page 16
Processed page 17
Processed page 18
Processed page 19
Processed page 20
Processed page 21
Processed page 22
Processed page 23
Processed page 24
Processed page 25
Processed page 26
Processed page 27
Processed page 28
Processed page 29
Processed page 30
Processed page 31
Processed page 32
Processed page 33
Processed page 34
Processed page 35
Processed page 36
Processed page 37
Processed page 38
Processed page 39
Processed page 40
Processed page 41
Processed page 42
Processed page 43
Processed page 44
Processed page 45
Processed page 46
Processed page 47
Processed page 48
Processed page 49
Processed page 50
Processed page 51
Processed page 52
Processed page 53
Processed page 54
Processed page 55
Processed page 56
P

In [11]:
 # Define the path for the output file
output_file_path = 'collected_links.txt'

# Open the file in write mode
with open(output_file_path, 'w') as file:
    for link in all_links:
        file.write(link + '\n')  # Write each link on a new line

print(f"All links have been saved to {output_file_path}")


All links have been saved to collected_links.txt


### SCRAPE DE INFO 

In [3]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options
import logging
import os

In [2]:
service = Service(executable_path='./chromedriver-linux64/chromedriver')
driver = webdriver.Chrome(service=service)

In [12]:
# Define the path for the input file containing links
input_file_path = 'collected_links.txt'

# Define the path for the output CSV file
output_file_path = 'extracted_data.csv'

# Define the path for the progress file
progress_file_path = 'progress.txt'

In [13]:
def extract_data_from_url(url):
    driver.get(url)
    time.sleep(3)  

    headword_element = driver.find_element(By.CSS_SELECTOR, "h1.oxencycl-title span.oxencycl-headword")
    headword = headword_element.text

    frequency_element = driver.find_element(By.CSS_SELECTOR, "h1.oxencycl-title span.headwordInfo")
    frequency = frequency_element.text.split(':')[-1].strip()  

    sub_divs = driver.find_elements(By.CSS_SELECTOR, "div.bodyCssRoot div.div1")
    description = "\n".join([div.text for div in sub_divs])

    return headword, frequency, description

In [14]:
with open(input_file_path, 'r') as file:
    links = [line.strip() for line in file.readlines()]

In [15]:
start_index = 0

In [16]:
# Read the progress file to get the last processed link
if os.path.exists(progress_file_path):
    with open(progress_file_path, 'r') as file:
        last_processed_link = file.read().strip()
        if last_processed_link in links:
            start_index = links.index(last_processed_link) + 1

In [17]:
# Open the CSV file for writing (append mode)
with open(output_file_path, 'a', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Link', 'Headword', 'Frequency', 'Description']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    if start_index == 0:
        writer.writeheader()  # Write header only if starting from the beginning

    # Iterate through each link starting from the last processed link
    for link in tqdm(links[start_index:], desc="Processing links"):
        try:
            headword, frequency, description = extract_data_from_url(link)
            writer.writerow({'Link': link, 'Headword': headword, 'Frequency': frequency, 'Description': description})
            logging.info(f"Extracted data from {link}")

            # Write the last processed link to the progress file
            with open(progress_file_path, 'w') as progress_file:
                progress_file.write(link)
        except Exception as e:
            logging.error(f"Failed to extract data from {link}: {e}")
            # Restart the WebDriver in case of an exception
            # driver.quit()
            # driver = create_webdriver()

# Close the browser
driver.quit()
logging.info("All data has been saved to extracted_data.csv")

Processing links:   0%|          | 3/83340 [00:13<99:16:29,  4.29s/it] 

In [10]:
# with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
#     fieldnames = ['Link', 'Headword', 'Frequency', 'Description']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     for link in tqdm(links, desc="Processing links"):
#         try:
#             headword, frequency, description = extract_data_from_url(link)
#             writer.writerow({'Link': link, 'Headword': headword, 'Frequency': frequency, 'Description': description})
#             # print(f"Extracted data from {link}")
#         except Exception as e:
#             print(f"Failed to extract data from {link}: {e}")

# driver.quit()
# print(f"All data has been saved to {output_file_path}")